## Deliverable 2. Create a Customer Travel Destinations Map.

In [51]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import time

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [52]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,78.96,73,40,6.91,scattered clouds
1,1,Mikuni,JP,36.2171,136.1519,71.51,84,15,15.01,few clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,29.86,100,90,25.32,snow
3,3,Katangli,RU,51.7102,143.2326,44.94,69,84,12.10,broken clouds
4,4,Launceston,AU,-41.4500,147.1667,46.15,78,1,1.99,clear sky


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 75
What is your desired maximum temperature for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,78.96,73,40,6.91,scattered clouds
6,6,Rikitea,PF,-23.1203,-134.9692,76.80,75,46,21.39,scattered clouds
23,23,Samana,DO,19.2056,-69.3369,80.56,82,23,8.14,few clouds
27,27,Tual,ID,-5.6667,132.7500,81.00,80,84,20.42,broken clouds
33,33,Thinadhoo,MV,0.5333,72.9333,83.52,73,98,18.75,overcast clouds
35,35,Ahuimanu,US,21.4447,-157.8378,83.91,55,75,11.50,broken clouds
36,36,Waingapu,ID,-9.6567,120.2641,79.72,78,93,14.38,overcast clouds
40,40,Kavieng,PG,-2.5744,150.7967,84.43,72,88,12.01,overcast clouds
46,46,Nishihara,JP,26.1842,127.7558,82.00,94,75,4.61,broken clouds
47,47,Tutoia,BR,-2.7619,-42.2744,79.48,81,22,8.43,few clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                219
City                   219
Country                219
Lat                    219
Lng                    219
Max Temp               219
Humidity               219
Cloudiness             219
Wind Speed             219
Current Description    219
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,78.96,73,40,6.91,scattered clouds
6,6,Rikitea,PF,-23.1203,-134.9692,76.80,75,46,21.39,scattered clouds
23,23,Samana,DO,19.2056,-69.3369,80.56,82,23,8.14,few clouds
27,27,Tual,ID,-5.6667,132.7500,81.00,80,84,20.42,broken clouds
33,33,Thinadhoo,MV,0.5333,72.9333,83.52,73,98,18.75,overcast clouds


In [31]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = None
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,None
6,Rikitea,PF,76.80,scattered clouds,-23.1203,-134.9692,None
23,Samana,DO,80.56,few clouds,19.2056,-69.3369,None
27,Tual,ID,81.00,broken clouds,-5.6667,132.7500,None
33,Thinadhoo,MV,83.52,overcast clouds,0.5333,72.9333,None


In [35]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")
    time.sleep(1)

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [36]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
6,Rikitea,PF,76.80,scattered clouds,-23.1203,-134.9692,Pension Maro'i
23,Samana,DO,80.56,few clouds,19.2056,-69.3369,The Bannister - Hotel & Yacht Club
27,Tual,ID,81.00,broken clouds,-5.6667,132.7500,Grand Vilia Hotel
35,Ahuimanu,US,83.91,broken clouds,21.4447,-157.8378,Paradise Bay Resort


In [38]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [45]:
clean_hotel_df["joint"] = clean_hotel_df["Current Description"] +" and " + clean_hotel_df["Max Temp"].astype(str)

C:\Users\fbrum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [49]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{joint} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [50]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))